In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
#df = pd.read_csv('usable_sp2019.csv', index_col=0)

In [15]:
# predictors
predictors = ["Close", "Open", "High", "Low", "Prev 1", "Prev 2","Prev 3","Prev 4","Prev 5","Prev 6","Prev 7",
              "Prev 8","Prev 9","Prev 10"]

In [21]:
import pickle

In [2]:
#filename = 'base_tree.sav'

In [3]:
# change model threshold
#model = pickle.load(open(filename, 'rb'))

In [40]:
# train model
from sklearn.ensemble import RandomForestClassifier

def train_model(train, predictors, nest=1000, minsamp=50):
    model = RandomForestClassifier(n_estimators=nest, min_samples_split=minsamp, random_state=1)
    model.fit(train[predictors], train["Target"])
    return model 

In [41]:
# predict with model with customizable threshold
def predict(test, predictors, model, threshold=0.5):
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >= threshold] = 1
    preds[preds < threshold] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [52]:
# backtest model
from sklearn.metrics import precision_score

def backtest(model, predictors, test, threshold=0.5):
    preds = predict(test, predictors, model, threshold)
    combined = pd.concat([test["Target"], preds], axis=1)
    print("Accuracy: ", precision_score(preds["Target"], preds["Predictions"]))
    print("Test distribution: ")
    print(preds["Target"].value_counts() / preds.shape[0])
    return combined

In [59]:
# split train and test data
def split_train_test(df, frac=0.8):
    train = df.sample(frac=frac)
    test = df.drop(train.index)
    train = train.reset_index()
    test = test.reset_index()
    del train['index']
    del test['index']
    return train, test